# MovieLens Dataset example
https://grouplens.org/datasets/movielens/

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.append("/home/ubuntu/rankability_toolbox")

In [13]:
import pandas as pd
import numpy as np
import dill

In [22]:
import pyrankability

In [23]:
DATA_DIR='ml-latest-small'

## Read and preprocess the data

In [24]:
links = pd.read_csv("%s/links.csv"%DATA_DIR)
display(links.head())
links["tmdbId"] = links["tmdbId"].fillna(-1).astype(int)
display(links.head())
display(links.dtypes)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


movieId    int64
imdbId     int64
tmdbId     int64
dtype: object

In [25]:
movies = pd.read_csv("%s/movies.csv"%DATA_DIR)
display(movies.head())
movies["genres"] = movies["genres"].str.split("|")
display(movies.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [26]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

movies = movies.join(pd.DataFrame(mlb.fit_transform(movies['genres']),
                          columns=mlb.classes_,
                          index=movies.index))


In [27]:
movies

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),[Comedy],0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),"[Action, Animation, Comedy, Fantasy]",0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,No Game No Life: Zero (2017),"[Animation, Comedy, Fantasy]",0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint (2017),[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),"[Action, Animation]",0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
ratings = pd.read_csv("%s/ratings.csv"%DATA_DIR)
display(ratings.head())

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Exploratory

**Number of unique movies along with their counts**

In [29]:
ratings.set_index("movieId").join(movies.set_index("movieId"))["title"].value_counts()

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
                                   ... 
Oscar (1991)                          1
Super, The (1991)                     1
Merry Madagascar (2009)               1
Lust for Life (1956)                  1
Geostorm (2017)                       1
Name: title, Length: 9719, dtype: int64

**What if we say 50 people in total had to rank a movie to even start considering it?**

In [30]:
counts = ratings.set_index("movieId").join(movies.set_index("movieId"))["title"].value_counts()
counts[counts > 50]

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
                                   ... 
The Devil's Advocate (1997)          51
Army of Darkness (1993)              51
Mulholland Drive (2001)              51
Training Day (2001)                  51
Bad Boys (1995)                      51
Name: title, Length: 437, dtype: int64

**This gets us to 437 movies, so that's a decent D matrix I think.**

For a start, let's fill in values in D by finding users who ranked two movies and then storing the difference in rating.

In [31]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
from itertools import combinations
flatten = lambda l: [item for sublist in l for item in sublist]
df1 = ratings.groupby("userId").apply(lambda df: 
                                     pd.DataFrame([flatten(tup) for tup in list(combinations(df.values,2))],columns=[v+"_i"for v in df.columns]+[v+"_j"for v in df.columns],dtype=int).
                                     set_index(["movieId_i","movieId_j"]))

df2 = df1.apply(lambda x: x["rating_i"]-x["rating_j"],axis=1).unstack()
display(df2)
#fix negatives when square
#inxs = np.where(df2 < 0)
#print("Fix negative numbers",len(inxs[0]))
#df2.values[inxs[1],inxs[0]] = -df2.values[inxs[0],inxs[1]]
#df2.values[inxs[0],inxs[1]] = 0
#display(df2.groupby().apply(lambda x: x.unstack()))
#inxs = np.where(df2 < 0)
#print("After fix negative numbers",len(inxs[0]))



In [ ]:
means = df2.stack().groupby(["movieId_i","movieId_j"]).mean()

In [ ]:
counts = df2.stack().groupby(["movieId_i","movieId_j"]).count()

**A quick glance at our counts and means**

In [ ]:
display(counts.head())

In [ ]:
display(counts.head())

## Sparse format to D
We now have a parse format, but we need to turn this into a D matrix. Let's only use a movie if there exists a paired count greater than 10.

In [ ]:
count_mask = counts > 10
print(len(np.unique(list(counts[count_mask].unstack().index) + list(counts[count_mask].unstack().columns))))

This means we would have a D matrix that is size 2021 by 2021. A little larger than our target of 500 x 500. But this is still small enough that we can construct the total matrix.

In [ ]:
D_counts = counts[count_mask].unstack()
D_means = means[count_mask].unstack()

In [ ]:
D_counts.stack()

We need to label the data so we can search for specific genres

In [ ]:
D_counts_labelled = D_counts.join(movies.set_index("movieId")).transpose().join(movies.set_index("movieId"))
D_means_labelled = D_means.join(movies.set_index("movieId")).transpose().join(movies.set_index("movieId"))

**Break the data into genres**

In [ ]:
D_counts_labelled

In [ ]:
genres = ["Comedy","Romance"]
genre = genres[1]

In [ ]:
from scipy.sparse import csr_matrix

i_mask = D_counts_labelled.loc[genre] == 1
j_mask = D_counts_labelled.loc[:,genre] == 1
D_means_genre = D_means.loc[i_mask,j_mask]
D_counts_genre = D_counts.loc[i_mask,j_mask]
inxs = np.where(~D_means_genre.isna())
n = max([max(inxs[0]),max(inxs[1])])+1
row = np.ix_(inxs[0],inxs[1])[0].reshape(1,-1)[0]
col = np.ix_(inxs[0],inxs[1])[1].reshape(1,-1)[0]
D = csr_matrix((D_means_genre.values[inxs].flatten(), (row,col)), shape=(n, n)).todense()
# Now fix negative numbers
inxs = np.where(D < 0)
print("Fix negative numbers",len(inxs[0]))
D[inxs[1],inxs[0]] = -D[inxs[0],inxs[1]]
D[inxs[0],inxs[1]] = 0
print("Just a subset of the matrix")
display(D[:10,:10])
inxs = np.where(D < 0)
print("After fix negative numbers",len(inxs[0]))
print("Summary of number of nonzero")
print(D.shape,(D>0).sum())

In [ ]:
print("hello")

In [ ]:
#k,details = pyrankability.hillside.count_lp(D)
#print(k,details["P"])

In [ ]:
#print(k,details["P"]) 

In [334]:
#pyrankability.hillside.objective_count_exhaustive(D[:8,:8])

(53,
 {'P': [(6, 2, 1, 4, 5, 7, 3, 0),
   (6, 2, 1, 4, 7, 3, 5, 0),
   (6, 2, 1, 4, 7, 5, 3, 0),
   (6, 2, 1, 5, 4, 7, 3, 0)]})